<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/qf-workshop-2021/introduction-to-machine-learning/blob/main/1_notebooks/introduction-to-machine-learning.ipynb">
        <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Supervised Models: Support Vector Machines (SVM)

## Support Vector Machines

Machine learning involves predicting and classifying data and to do so we employ various machine learning algorithms according to the dataset.
SVM or Support Vector Machine is a linear model for classification and regression problems. It can solve linear and non-linear problems and work well for many practical problems. This chapter will primarily focus on how SVM works as a classifier. It classifies data points into two classes at a time(this does not mean it is only a binary classifier, it only separates data points into two classes at a time), using a decision boundary (a hyperplane in this case). The primary objective of the Support vector Classifier is finding the ‘Optimal Separating Hyperplane(Decision Boundary)’. 

At first approximation what SVMs do is to find a separating line(or hyperplane) between data of two classes. SVM is an algorithm that takes the data as an input and outputs a line that separates those classes if possible.
Lets begin with a problem. Suppose you have a dataset as shown below and you need to classify the red rectangles from the blue ellipses(let’s say positives from the negatives). So your task is to find an ideal line that separates this dataset in two classes (say red and blue).

But, as you notice there isn’t a unique line that does the job. In fact, we have an infinite lines that can separate these two classes. So how does SVM find the ideal one???

#### SVM’s way to find the best line

According to the SVM algorithm we find the points closest to the line from both the classes.These points are called support vectors. Now, we compute the distance between the line and the support vectors. This distance is called the margin. Our goal is to maximize the margin. The hyperplane for which the margin is maximum is the optimal hyperplane.


Thus SVM tries to make a decision boundary in such a way that the separation between the two classes(that street) is as wide as possible.

### What is a Hyperplane?

In geometry, it is an n-dimensional generalization of a plane, a subspace with one less dimension(n-1) than its origin space. In one-dimensional space, it is a point, In two-dimensional space it is a line, In three-dimensional space, it is an ordinary plane, in four or more dimensional spaces, it is then called a ‘Hyperplane’. Take note of this, it is really how the Support Vector Machine works behind the scenes, the dimensions are the features represented in the data. For example, say we want to carry out a classification problem and we want to be able to tell if a product gets purchased or not(a binary classification), if there is just one feature(say Gender) available as a feature in the dataset, then it is in one-dimensional space and the subspace(the separating/decision boundary) representation will be (n-1=0) a 0-dimensional space(if I can call it that), represented with just a point showing the separation of classes(Purchased or not). If there are two features(Age and Gender), it is a two-dimensional space(2D), with either of Age and Gender on the X and y-axis, the decision boundary will be represented as a simple line. Similarly, if the features are three(Age, Gender, Income), the decision boundary will be a two-dimensional plane in a three-dimensional space(n-1). Furthermore, if we have a four or more dimensional space data points, then it is called a ‘Hyperplane’ with (n-1 dimension). 

It is important to note that the number of features for a given machine learning problem can be selected using a technique called ‘feature selection’, as not all features are necessarily useful, some can be redundant and create unnecessary noise in the data.



The Hyperplane is simply a concept that separates an n-dimensional space into two groups/halves. In machine learning terms, it is a form of a decision boundary that algorithms like the Support Vector Machine uses to classify or separate data points. There are two parts to it, the negative side hyperplane and the positive part hyperplane, where data points/instances can lie on either part, signifying the group/class they belong to.




We are going to use $\mathbf{w}$ the vector that is orthogonal to the hyperplane ...

### Hard Margins

### Lagrange Multiplier

Because we have a contrained optimization problem (with inequality contraints) we can recast the problem using Lagrange multipliers, $\alpha \ge 0$ and search for the critical points of 

\begin{equation}\label{eq:7.7}
L = \frac{1}{2} \vert \theta \vert^2 - \sum\limits_{n=1}^N \alpha_n \left( y^{(n)} \left( \mathbf{\theta}^T \mathbf{x}^{(n)} + \theta_0 \right) -1 \right)
\end{equation}

To find the critical points we differentiate with respect to the scalar $\theta_0$ and with respect to the vector $\mathbf{\theta}$. Remember that differentiation with respect to a vector just means differentiating with respect to each of its entries. Setting these derivatives to zero you end up with:

\begin{equation}\label{eq:7.8}
\sum\limits_{n=1}^N \alpha_n  y^{(n)} = 0
\end{equation}

and 

\begin{equation}\label{eq:7.9}
\mathbf{\theta} = \sum\limits_{n=1}^N \alpha_n  y^{(n)} \mathbf{x}^{(n)} 
\end{equation}

which means that our vector orthogonal to the hyperplane is just a linear combination of sample vectors. Substituting \eqref{eq:7.9} into \eqref{eq:7.7}  and using \eqref{eq:7.8} results in 

\begin{equation}
L = \sum\limits_{n=1}^N \alpha_n - \frac{1}{2} \sum\limits_{i=1}^N \sum\limits_{j=1}^N \alpha_i \alpha_j y^{(i)} 
{\mathbf{x}^{(i)}}^T \mathbf{x}^{(j)} 
\end{equation}

We want to maximize $L$ over the $\alpha$s, all greater than or equal to zero, subject to \eqref{eq:7.8}. This is known as the dual problem.


Once we have found the $\alpha$s the dual version of the classifier for a new point $\mathbf{u}$ is the just
\begin{equation}
\sum\limits_{n=1}^N \alpna_n   y^{(n)} {\mathbf{x}^{(n)}}^T  \mathbf{u} + \theta_0
\end{equation}

whether this is greater or less than zero.

**TODO Chiarire bene la seguente frase:** The maximization of $L$ whill return almost all $\alpha$s as zero. Those that aren't zero correspond to the support vectors, the cases that define the margins.  

### Soft Margins



### Kernel Trick

We might have data that is not linearly separable but which nonetheless falls neatly into two groups, just not groups that can be divided by a hyperplane. A simple example would be that shown in Fig.

This data is clearly not linearly separable. We cannot draw a straight line that can classify this data. But, this data can be converted to linearly separable data in higher dimension. Lets add one more dimension and call it z-axis. For the data in Fig. we could go to three dimensions via the transformation

$$(x_1, x_2) \rightarrow (x_1, x_2, x_1^2 + x_2^2)$$

this would have the effect of lifting up the outer group of dots higher than the inner group in the new third dimension, making them linearly separable. Now the data is clearly linearly separable. Let the purple line separating the data in higher dimension be z=k, where k is a constant. Since, z=x²+y² we get x² + y² = k; which is an equation of a circle. So, we can project this linear separator in higher dimension back in original dimensions using this transformation. 

...

Thanks to the dual formulation in (-) we can see that finding the $\alpha$s depends only on the products ${\mathbf{x}^{(i)}}^T \mathbf{x}^{(j)}$, and it's easy to observe that also the classification of new data depends on the product ${\mathbf{x}^{(i)}}^T \mathbf{u}$. 

If we use $\Phi(\mathbf{x})$ to denote a transformation of the sample points (into higher dimensions) then the above observation show us that all we need to know for training and for classifying in the higher dimensional space are

$$\Phi\left({\mathbf{x}^{(i)}}\right)^T \Phi \left(\mathbf{x}^{(j)}\right)$$

and 

$$\Phi \left( {\mathbf{x}^{(i)}}\right)^T \Phi \bigl( \mathbf{u} \bigr) $$

for our transformed data. 

...



# Appendix. SVM Basic Terminology

**Vector**: This is simply the training examples/data points. It is also known as ‘Feature vectors’ in machine learning.

**Support + Vector**: This is simply a subset of the data closest to the hyperplane/decision boundary.

